In [73]:
# Importing Required Libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import cv2
import seaborn as sns
import matplotlib.pyplot as plt

In [51]:
# Reading Data from provided .csv file
data = pd.read_csv('./data/train.csv')
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB
None


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
# Retrieving labels from Data
labels = data['label']

# Converting into One-Hot Vectors for Classification
labels = pd.get_dummies(labels)
labels.shape

(42000, 10)

In [53]:
# Retrieving all pixel columns from Data
data = data.iloc[:,1:]
data.shape 

(42000, 784)

In [54]:
# Taking 40000 images to Train the model
train_data = data[2000:]
train_labels = labels[2000:]

train_data.shape, train_labels.shape

((40000, 784), (40000, 10))

In [55]:
# Taking 2000 images for Cross Validation
test_data = data[:2000]
test_labels = labels[:2000]

test_data.shape, test_labels.shape

((2000, 784), (2000, 10))

In [10]:
# Standard loc and scale values for kernel initializer 
def initialize_weights(shape, dtype=None):
    
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

# Standard loc and scale values for bias initializer
def initialize_bias(shape, dtype=None):
    
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

def DeepLearningModel(input_shape):
    model = Sequential()
    
    model.add(Input(input_shape))
    
    model.add(Dense(64, activation='relu', kernel_initializer=initialize_weights, bias_initializer=initialize_bias, kernel_regularizer=l2(3e-3)))
    
    model.add(Dense(128, activation='relu', kernel_initializer=initialize_weights, bias_initializer=initialize_bias, kernel_regularizer=l2(3e-3)))

    model.add(Dense(256, activation='relu', kernel_initializer=initialize_weights, bias_initializer=initialize_bias, kernel_regularizer=l2(3e-3)))
                
    model.add(Dense(10, activation='softmax', kernel_initializer=initialize_weights, bias_initializer=initialize_bias, kernel_regularizer=l2(3e-3)))    
    
    return model


model = DeepLearningModel((784))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                50240     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
Total params: 94,154
Trainable params: 94,154
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Hyperparameters
lr = 0.0003
epochs = 50
batch_size = 128

optimizer = Adam(lr)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size)

Epoch 1/50
313/313 [==============================] - 1s 3ms/step - loss: 0.9176 - accuracy: 0.7241
Epoch 2/50
313/313 [==============================] - 1s 3ms/step - loss: 0.2472 - accuracy: 0.9362
Epoch 3/50
313/313 [==============================] - 1s 3ms/step - loss: 0.1838 - accuracy: 0.9572
Epoch 4/50
313/313 [==============================] - 1s 2ms/step - loss: 0.1552 - accuracy: 0.9664
Epoch 5/50
313/313 [==============================] - 1s 2ms/step - loss: 0.1315 - accuracy: 0.9732
Epoch 6/50
313/313 [==============================] - 1s 2ms/step - loss: 0.1248 - accuracy: 0.9752
Epoch 7/50
313/313 [==============================] - 1s 2ms/step - loss: 0.1130 - accuracy: 0.9802
Epoch 8/50
313/313 [==============================] - 1s 2ms/step - loss: 0.1083 - accuracy: 0.9804
Epoch 9/50
313/313 [==============================] - 1s 2ms/step - loss: 0.0964 - accuracy: 0.9850
Epoch 10/50
313/313 [==============================] - 1s 3ms/step - loss: 0.0940 - accuracy: 0.9866

In [12]:
# Evaluating model using Validation Data
model.evaluate(test_data, test_labels)

63/63 [==============================] - 0s 775us/step - loss: 0.1403 - accuracy: 0.9700


[0.14025822281837463, 0.9700000286102295]

In [13]:
# Reading Test Dataset
test = pd.read_csv('./data/test.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB


In [14]:
# Making Predictions on existing model.
prediction = model.predict(test)

# Rounding the prediction and converting float64 output to int64 output.
prediction = np.array(np.round(prediction), dtype='int64')

# Converting one-hot encoding into label encoding
prediction = (np.argmax(prediction, axis=1)).reshape(-1, 1)

# Creating a DataFrame similar to Example Submission
out = [{'ImageId': i+1, 'Label': prediction[i][0]} for i in range(len(prediction))]

# Creating a .csv file from the DataFrame
pd.DataFrame(out).to_csv('submission.csv', index=False)

#### Achieved 97% accuracy with Deep Learning Classifier

Now let's see how a Convolutional Neural Network will perform on the Dataset.

In [56]:
imgdata = np.reshape(np.array(data),(data.shape[0],28,28,1))
imglabels = labels

imgdata.shape, imglabels.shape

((42000, 28, 28, 1), (42000, 10))

In [57]:
# Taking 40000 images to Train the model
img_train_data = imgdata[2000:]
img_train_labels = imglabels[2000:]

img_train_data.shape, img_train_labels.shape

((40000, 28, 28, 1), (40000, 10))

In [58]:
# Taking 2000 images for Cross Validation
img_test_data = imgdata[:2000]
img_test_labels = imglabels[:2000]

img_test_data.shape, img_test_labels.shape

((2000, 28, 28, 1), (2000, 10))

In [74]:
# Building an Image Dataset for a Convolutional Neural Network.
def convolutional_model(input_shape):
    model = Sequential()
    
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))
    
    model.add(MaxPooling2D())
    
    model.add(Dropout(0.25))
    
    model.add(Conv2D(32, (3,3), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    
    model.add(MaxPooling2D())
    
    model.add(Dropout(0.25))
    
    model.add(Conv2D(32, (3,3), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    
    model.add(Flatten())
    
    model.add(Dense(1024, activation='relu',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    model.add(Dense(10, activation='softmax',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    return model

In [78]:
model = convolutional_model((28,28,1))
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 5, 5, 32)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 3, 3, 32)        

In [79]:
# Hyperparameters
lr = 0.0003
epochs = 10
batch_size = 128

optimizer = Adam(lr)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(img_train_data, img_train_labels, epochs=epochs, batch_size=batch_size)

Epoch 1/10
313/313 [==============================] - 11s 34ms/step - loss: 1.0601 - accuracy: 0.6510
Epoch 2/10
313/313 [==============================] - 12s 38ms/step - loss: 0.1498 - accuracy: 0.9639
Epoch 3/10
313/313 [==============================] - 12s 39ms/step - loss: 0.1204 - accuracy: 0.9711
Epoch 4/10
313/313 [==============================] - 11s 35ms/step - loss: 0.0984 - accuracy: 0.9784
Epoch 5/10
313/313 [==============================] - 11s 34ms/step - loss: 0.0864 - accuracy: 0.9814
Epoch 6/10
313/313 [==============================] - 12s 37ms/step - loss: 0.0801 - accuracy: 0.9831
Epoch 7/10
313/313 [==============================] - 12s 38ms/step - loss: 0.0709 - accuracy: 0.9859
Epoch 8/10
313/313 [==============================] - 12s 39ms/step - loss: 0.0707 - accuracy: 0.9855
Epoch 9/10
313/313 [==============================] - 15s 47ms/step - loss: 0.0679 - accuracy: 0.9863
Epoch 10/10
313/313 [==============================] - 17s 55ms/step - loss: 0.064

In [80]:
model.evaluate(img_test_data, img_test_labels)

63/63 [==============================] - 0s 3ms/step - loss: 0.0547 - accuracy: 0.9910


[0.05468718707561493, 0.9909999966621399]

In [81]:
# Reading Test Dataset
test = pd.read_csv('./data/test.csv')
test = np.reshape(np.array(test), (test.shape[0], 28, 28, 1))

In [82]:
# Making Predictions on existing model.
prediction = model.predict(test)

# Rounding the prediction and converting float64 output to int64 output.
prediction = np.array(np.round(prediction), dtype='int64')

# Converting one-hot encoding into label encoding
prediction = (np.argmax(prediction, axis=1)).reshape(-1, 1)

# Creating a DataFrame similar to Example Submission
out = [{'ImageId': i+1, 'Label': prediction[i][0]} for i in range(len(prediction))]

# Creating a .csv file from the DataFrame
pd.DataFrame(out).to_csv('submission_v2.csv', index=False)